# Полносвязная нейронная сеть прямого распространения

Полносвязная нейронная сеть прямого распространения, часто называемая многослойным персептроном (MLP), представляет собой тип архитектуры искусственной нейронной сети. Это одна из наиболее базовых и широко используемых моделей нейронных сетей. Термин «полносвязная» означает, что каждый нейрон в одном слое соединен с каждым нейроном в следующем слое.

Первый слой сети – это **входной слой**. Он содержит нейроны, которые представляют входные признаки или данные. Каждый нейрон в этом слое соответствует признаку или атрибуту входных данных.

Между входным и выходным слоями может быть один или несколько **скрытых слоев**. Каждый нейрон в скрытом слое полностью связан со всеми нейронами в предыдущих и последующих слоях. Эти скрытые слои позволяют сети изучать сложные закономерности и взаимосвязи в данных.

Каждое соединение между нейронами имеет связанный с ним **вес**, который определяет силу этого соединения. Кроме того, у каждого нейрона есть **смещение**. Сеть обучается оптимальным весам и смещениям в процессе обучения для минимизации ошибки в прогнозах.

Нейроны в скрытых слоях и выходном слое обычно используют **функции активации** для введения нелинейности в сеть. Общие функции активации включают сигмоиду, гиперболический тангенс (tanh) и линейное выпрямление (ReLU).

Сеть вычисляет свой выход, передавая данные от входного слоя через скрытые слои к выходному слою. Этот процесс называется **прямым распространением**. Выход каждого нейрона в слое определяется путем сложения взвешенных входов.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [2]:
device

device(type='cpu')

## 1. Данные

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


## Нейронная сеть

Строим модель нейронной сети с использованием контейнера `nn.Sequential`. Эта сеть состоит из трёх слоев:

1. `nn.Linear(784, 200)` представляет собой входной слой. Он содержит 784 входных нейрона (признаков) и 200 выходных нейронов. Слой `nn.Linear` применяет линейное преобразование (матричное умножение) к входным данным.

2. `nn.Tanh()` представляет собой скрытый слой с активационной функцией гиперболического тангенса (Tanh), примененной к 200 выходным нейронам из предыдущего слоя. Функция Tanh вводит нелинейность в сеть.

3. `nn.Linear(200, 100), nn.Sigmoid()` представляет собой ещё один полностью связанный скрытый слой с 100 нейронами, применяется активационная функция сигмоида

4. `nn.Linear(200, 10)` представляет собой выходной слой. Он берет 200 нейронов из предыдущего слоя и отображает их на 10 выходных нейронов, которые могут использоваться для задач классификации или регрессии

In [4]:
#[784, 200, 100, 10]
model = nn.Sequential(nn.Linear(784, 200), nn.Tanh(), 
                      nn.Linear(200, 100), nn.Sigmoid(), 
                      nn.Linear(200, 10))

In [5]:
model

Sequential(
  (0): Linear(in_features=784, out_features=200, bias=True)
  (1): Tanh()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=200, out_features=10, bias=True)
)

Создаём тензор W, заполненный нулями. Он имеет размерность 10x100, что соответствует форме весов пятого слоя нашей модели.

`model[4].weight.data = W`: заменяем веса пятого слоя на нули.

Это может быть полезно в тех случаях, когда мы хотим инициализировать веса конкретного слоя в нашей нейронной сети определенным значением. Инициализация весов – это важный аспект обучения нейронной сети, и использование случайной инициализации (например, инициализация Xavier/Glorot или He) обычно обеспечивает более эффективное обучение сети.

In [6]:
W = torch.zeros(10, 100)
with torch.no_grad():
    model[4].weight.data = W

In [7]:
class MyNetwork(torch.nn.Module):   
    
    # Конструктор класса MyNetwork, который инициализирует объект класса. Он принимает четыре аргумента:
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        
        # вызывает конструктор родительского (базового) класса `torch.nn.Module`, 
        # чтобы инициализировать базовую функциональность PyTorch
        super().__init__()
        
        # Эти строки сохраняют переданные аргументы в виде атрибутов объекта класса, 
        # чтобы они были доступны во всех методах класса
        self.input_size = input_size     # размер входных данных
        self.hidden_size1 = hidden_size1 # размер скрытого слоя 1
        self.hidden_size2 = hidden_size2 # размер скрытого слоя 2
        self.num_classes = num_classes   # количество классов для задачи классификации
        
        # Компоненты нейронной сети:
        # два полносвязанных слоя fc1 и fc2, fc2 соединяет первый скрытый слой с вторым 
        # nn.Linear – слой, который выполняет линейное преобразование 
        # nn.ReLU() – активационная функция ReLU, применяемая после первого слоя
        self.fc1 = nn.Linear(input_size, hidden_size1) # fully connected
        self.func1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) # fully connected
        
        # Альтернативные компоненты сети. 
        # alt_fc1 – полносвязанный слой, а 
        # alt_func1 – активационная функция Tangent Hyperbolic (Tanh) 
        # Эти альтернативные компоненты могут использоваться в других ветвях сети или для экспериментов.
        self.alt_fc1 = nn.Linear(input_size, hidden_size2) #fully connected
        self.alt_func1 = nn.Tanh()

        # Оставшиеся компоненты сети
        # func2 – активационная функция Tanh, применяемая после второго скрытого слоя
        # fc3 – последний полносвязанный слой, который выводит результат в соответствии с задачей классификации 
        # на num_classes классов.
        self.func2 = nn.Tanh()
        self.fc3 = nn.Linear(hidden_size2, num_classes) # fully connected
        
    # Метод forward определяет, как данные проходят через нейронную сеть, x представляет собой входные данные
    def forward(self, x):
        
        # Данные проходят через первый полносвязанный слой fc1, а затем применяется активационная функция ReLU func1
        out = self.func1(self.fc1(x))
        # Данные проходят через второй полносвязанный слой fc2, а затем применяется активационная функция Tanh func2
        out = self.func2(self.fc2(out))
        
        # Альтернативная ветвь сети, где данные проходят через альтернативный полносвязанный слой alt_fc1 
        # и активационную функцию Tanh alt_func1
        out2 = self.alt_func1(self.alt_fc1(x))
        
        # Результаты из основной и альтернативной ветвей сети складываются. Затем объединенные данные проходят 
        # через последний полносвязанный слой fc3, который выдаст окончательный результат
        out = out + out2
        out = self.fc3(out)
        
        return out

In [8]:
model = MyNetwork(input_size=784, hidden_size1=200, hidden_size2=100, num_classes=10)

In [9]:
model

MyNetwork(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (func1): ReLU()
  (fc2): Linear(in_features=200, out_features=100, bias=True)
  (alt_fc1): Linear(in_features=784, out_features=100, bias=True)
  (alt_func1): Tanh()
  (func2): Tanh()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

In [10]:
x, y = train_dataset[0]
model(x.flatten())

tensor([ 0.0710, -0.0361,  0.1195,  0.1169, -0.2028, -0.0311, -0.0439, -0.2381,
        -0.0490, -0.0671], grad_fn=<ViewBackward0>)

In [11]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
#        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  
#        self.fc3 = nn.Linear(hidden_size2, hidden_size3)  
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
#        out = self.relu(self.fc2(out))
#        out = self.relu(self.fc3(out))
        out = self.fc2(out)
        
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [12]:
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

## 3. Лосс функция

In [13]:
# Loss
criterion = nn.CrossEntropyLoss()

## 4. optimizer и обучение

In [14]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
model = model.to(device) # Перемещение модели (или тензора) на указанное устройство (cuda или cpu)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0 # Количество правильно классифицированных примеров 
    total = 0   # Общее число примеров в тестовом наборе данных
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)   # reshape(-1, 28*28): каждый пакет изображений преобразуется 
                                                        # в плоский формат (с использованием -1, что означает, что 
                                                        # количество строк вычисляется на основе оставшихся размерностей)
        labels = labels.to(device)
        outputs = model(images)   # прогнозы модели для пакета тестовых изображений
        _, predicted = torch.max(outputs.data, 1)   # Вычисляются предсказанные метки, выбирая класс с наибольшей оценкой 
                                                    # из прогнозов модели. Массив predicted содержит предсказанные 
                                                    # метки для пакета данных
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.4587
Epoch [1/5], Step [200/600], Loss: 0.2711
Epoch [1/5], Step [300/600], Loss: 0.2100
Epoch [1/5], Step [400/600], Loss: 0.1220
Epoch [1/5], Step [500/600], Loss: 0.1404
Epoch [1/5], Step [600/600], Loss: 0.2637
Epoch [2/5], Step [100/600], Loss: 0.1789
Epoch [2/5], Step [200/600], Loss: 0.0698
Epoch [2/5], Step [300/600], Loss: 0.0384
Epoch [2/5], Step [400/600], Loss: 0.1659
Epoch [2/5], Step [500/600], Loss: 0.0778
Epoch [2/5], Step [600/600], Loss: 0.0669
Epoch [3/5], Step [100/600], Loss: 0.1103
Epoch [3/5], Step [200/600], Loss: 0.0796
Epoch [3/5], Step [300/600], Loss: 0.1210
Epoch [3/5], Step [400/600], Loss: 0.0722
Epoch [3/5], Step [500/600], Loss: 0.0380
Epoch [3/5], Step [600/600], Loss: 0.0746
Epoch [4/5], Step [100/600], Loss: 0.0466
Epoch [4/5], Step [200/600], Loss: 0.0998
Epoch [4/5], Step [300/600], Loss: 0.0357
Epoch [4/5], Step [400/600], Loss: 0.0250
Epoch [4/5], Step [500/600], Loss: 0.0480
Epoch [4/5], Step [600/600], Loss:

In [15]:
ss = [] # список для хранения размеров параметров (веса (весовые коэффициенты) и смещения (bias))
for p in model.parameters():
    ss.append(p.numel())    # Метод numel() возвращает количество элементов в параметре (например, количество чисел)
                            # Это количество элементов представляет собой размер параметра
print(ss)
sum(ss)

[392000, 500, 5000, 10]


397510

Список параметров `[392000, 500, 5000, 10]` представляет собой количество параметров (весов и смещений) для каждого слоя в нашей нейронной сети